# Amazon Database creation

This notebook's goal is to create a mock-up transactionnal database for a cosmetics e-commerce.
The data is based on a dataset from amazon reviews.

In [1]:
!pip install pandas sqlalchemy psycopg2-binary faker datasets transformers dotenv


## Data schema creation

For the first version of the production database we are using neon db, the connection information is contained in an .env file.

Important note: All dataframe objects to be inserted in the db will be called `"df_someting"` and all objects read from the database will be called `"db_something`.

In [113]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os

# the connection string for the database
db_connection_string = os.getenv("sql_connection_string")
db_user = os.getenv("db_user")
db_pw = os.getenv("db_pw")
db_host = os.getenv("db_host")
db_name = os.getenv("db_name")


In [4]:
# create_schema.py

import psycopg2
from psycopg2 import sql

# 1) Install with: pip install psycopg2-binary

# Database connection parameters
DB_CONFIG = {
    "host":     db_host,       # your host
    "port":     5432,              # PostgreSQL default port
    "dbname":   db_name,   # your database name
    "user":     db_user,   # your username
    "password": db_pw,   # your password
}

### DB creation

In [51]:
# SQL statements to run (e.g. CREATE TABLE)
# Load SQL script from file
with open("db_creation.sql", "r") as file:
    ddl = file.read()
def run_schema_queries():
    conn = None
    try:
        # Connect
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()

        # Execute all queries
        cur.execute(ddl)
        print("✅ Executed:\n", ddl.strip(), "\n")

        # Persist
        conn.commit()
        print("All schema objects created successfully.")

    except Exception as e:
        print("❌ Error:", e)

    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    run_schema_queries()


✅ Executed:
 -- CUSTOMER TABLE
CREATE TABLE CUSTOMER (
    C_ID   VARCHAR(40),
    FNAME  VARCHAR(30) NOT NULL,
    LNAME  VARCHAR(30) NOT NULL,
    PHONE  CHAR(10) UNIQUE NOT NULL,
    EMAIL  VARCHAR(60) UNIQUE NOT NULL,
    PWD    VARCHAR(60) NOT NULL,
    PRIMARY KEY (C_ID)
);

-- SHIPPING_DETAILS TABLE
CREATE TABLE SHIPPING_DETAILS (
    ADDRESS_ID INTEGER GENERATED BY DEFAULT AS IDENTITY,
    STREET_ADDRESS VARCHAR(100) NOT NULL,
    CITY VARCHAR(100) NOT NULL,
    STATE VARCHAR(100) NOT NULL,
    ZIP VARCHAR(10) NOT NULL,
    COUNTRY VARCHAR(60) NOT NULL,
    PHONE CHAR(10) NOT NULL,
    PRIMARY KEY (ADDRESS_ID)
);

-- CUSTOMER_SHIPPING TABLE
CREATE TABLE CUSTOMER_SHIPPING (
    ADDRESS_ID INTEGER,
    C_ID VARCHAR(40),
    IS_DEFAULT CHAR(1) DEFAULT '0' CHECK (IS_DEFAULT IN ('0', '1')),
    PRIMARY KEY (ADDRESS_ID, C_ID),
    CONSTRAINT CSADDRESSFK FOREIGN KEY (ADDRESS_ID) REFERENCES SHIPPING_DETAILS (ADDRESS_ID) ON DELETE CASCADE,
    CONSTRAINT CSCUSTOMERFK FOREIGN KEY (C_ID) 

### Implement procedures

In [116]:
# SQL statements to run (e.g. CREATE TABLE)
# Load SQL script from file
with open("Procedures.sql", "r") as file:
    ddl = file.read()
def run_schema_queries():
    conn = None
    try:
        # Connect
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()

        # Execute all queries
        cur.execute(ddl)
        print("✅ Executed:\n", ddl.strip(), "\n")

        # Persist
        conn.commit()
        print("All schema objects created successfully.")

    except Exception as e:
        print("❌ Error:", e)

    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    run_schema_queries()


✅ Executed:
 CREATE OR REPLACE PROCEDURE APPLY_DAILY_DEALS(
    IN cart_id INTEGER,
    IN o_date DATE,
    IN minimum_price NUMERIC,
    INOUT total NUMERIC
)
LANGUAGE plpgsql
AS $$
DECLARE
    this_product RECORD;
    discount_percent NUMERIC := 0;
    item_price NUMERIC;
    item_discount NUMERIC;
BEGIN
    FOR this_product IN
        SELECT * FROM cart_items WHERE cart_id = cart_id
    LOOP
        BEGIN
            SELECT discount INTO discount_percent
            FROM daily_deals
            WHERE p_id = this_product.p_id
              AND deal_date = o_date;
        EXCEPTION
            WHEN no_data_found THEN
                discount_percent := 0;
        END;

        SELECT price INTO item_price
        FROM product
        WHERE p_id = this_product.p_id;

        FOR i IN 1..this_product.qty LOOP
            item_discount := (discount_percent / 100.0) * item_price;
            total := total - item_discount;
        END LOOP;

        IF total < minimum_price THEN
         

### Set up triggers

In [117]:
# SQL statements to run (e.g. CREATE TABLE)
# Load SQL script from file
with open("Procedures.sql", "r") as file:
    ddl = file.read()
def run_schema_queries():
    conn = None
    try:
        # Connect
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()

        # Execute all queries
        cur.execute(ddl)
        print("✅ Executed:\n", ddl.strip(), "\n")

        # Persist
        conn.commit()
        print("All schema objects created successfully.")

    except Exception as e:
        print("❌ Error:", e)

    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    run_schema_queries()


✅ Executed:
 CREATE OR REPLACE PROCEDURE APPLY_DAILY_DEALS(
    IN cart_id INTEGER,
    IN o_date DATE,
    IN minimum_price NUMERIC,
    INOUT total NUMERIC
)
LANGUAGE plpgsql
AS $$
DECLARE
    this_product RECORD;
    discount_percent NUMERIC := 0;
    item_price NUMERIC;
    item_discount NUMERIC;
BEGIN
    FOR this_product IN
        SELECT * FROM cart_items WHERE cart_id = cart_id
    LOOP
        BEGIN
            SELECT discount INTO discount_percent
            FROM daily_deals
            WHERE p_id = this_product.p_id
              AND deal_date = o_date;
        EXCEPTION
            WHEN no_data_found THEN
                discount_percent := 0;
        END;

        SELECT price INTO item_price
        FROM product
        WHERE p_id = this_product.p_id;

        FOR i IN 1..this_product.qty LOOP
            item_discount := (discount_percent / 100.0) * item_price;
            total := total - item_discount;
        END LOOP;

        IF total < minimum_price THEN
         

## Read-Only user creation

In [108]:
import psycopg2

student_pwd = os.getenv("student_pwd")


sql = f"""
CREATE USER readonly_user WITH PASSWORD '{student_pwd}';
GRANT CONNECT ON DATABASE neondb TO readonly_user;
GRANT USAGE ON SCHEMA public TO readonly_user;
GRANT SELECT ON ALL TABLES IN SCHEMA public TO readonly_user;
ALTER DEFAULT PRIVILEGES IN SCHEMA public GRANT SELECT ON TABLES TO readonly_user;
"""

conn = psycopg2.connect(**DB_CONFIG
)

cur = conn.cursor()
cur.execute(sql)
conn.commit()
cur.close()
conn.close()


In [112]:
read_only_user_connection = os.getenv("read_only_user_connection_str")

## test the read_only connection

# let's import the customer table from the db
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(read_only_user_connection)
with engine.connect() as conn:
    db_customer = pd.read_sql_table(table_name="customer",con=conn)

db_customer.head()

,c_id,fname,lname,phone,email,pwd
0,AE222BBOVZIF42YOOPNBXL4UUMYA,Connor,MASON,0604823672,connor.mason99@test.com.org,qOkVgJWdR9rk
1,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,Calvin,ANDERSON,0409711663,calvin.anderson6@example.com,uqt7PUOyH4TS
2,AE222X475JC6ONXMIKZDFGQ7IAUA,William,ANDREWS,0109386339,william.andrews27@test.com.org,4CHPsiqjjhh4
3,AE222Y4WTST6BUZ4J5Y2H6QMBITQ,Antonio,COLLINS,0778653405,antonio.collins80@example.org,AytJdxIZAaoA
4,AE2232TEZOEWQLAFEX2NA6VBGMYQ,Robert,MILLS,0526056936,robert.mills52@example.org,VNXPKkKnB1Os


## Insert data

Now that the db has been created we need to populate it with data.
We'll base ouserlves off of a cosmetics product review database from Amazon.

In [5]:
import datasets
datasets.logging.set_verbosity_error()

/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from datasets import load_dataset

dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)

In [7]:
dataset["full"]

Dataset({
    features: ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'],
    num_rows: 701528
})

In [8]:
metadata = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_All_Beauty", split="full", trust_remote_code=True)

In [9]:
metadata[0]

{'main_category': 'All Beauty',
 'title': 'Howard LC0008 Leather Conditioner, 8-Ounce (4-Pack)',
 'average_rating': 4.8,
 'rating_number': 10,
 'features': [],
 'description': [],
 'price': 'None',
 'images': {'hi_res': [None,
   'https://m.media-amazon.com/images/I/71i77AuI9xL._SL1500_.jpg'],
  'large': ['https://m.media-amazon.com/images/I/41qfjSfqNyL.jpg',
   'https://m.media-amazon.com/images/I/41w2yznfuZL.jpg'],
  'thumb': ['https://m.media-amazon.com/images/I/41qfjSfqNyL._SS40_.jpg',
   'https://m.media-amazon.com/images/I/41w2yznfuZL._SS40_.jpg'],
  'variant': ['MAIN', 'PT01']},
 'videos': {'title': [], 'url': [], 'user_id': []},
 'store': 'Howard Products',
 'categories': [],
 'details': '{"Package Dimensions": "7.1 x 5.5 x 3 inches; 2.38 Pounds", "UPC": "617390882781"}',
 'parent_asin': 'B01CUPMQZE',
 'bought_together': None,
 'subtitle': None,
 'author': None}

In [57]:
# Create the list of user ids to insert in the db
import numpy as np
c_ids = np.unique(dataset["full"]["user_id"])[:100000]
c_ids

array(['AE222BBOVZIF42YOOPNBXL4UUMYA', 'AE222FP7YRNFCEQ2W3ZDIGMSYTLQ',
       'AE222X475JC6ONXMIKZDFGQ7IAUA', ...,
       'AEO7LHP34RKZ4BZMFLUSCN3ZKDFQ', 'AEO7LJ5DKX4CCINGZD3L74KP7SCQ',
       'AEO7LX7MRZMII5QOEQT7UUR5AIKA'], shape=(100000,), dtype='<U36')

### Customers

```sql
-- CUSTOMER TABLE
CREATE TABLE CUSTOMER (
    C_ID   VARCHAR(40),
    FNAME  VARCHAR(30) NOT NULL,
    LNAME  VARCHAR(30) NOT NULL,
    PHONE  CHAR(10) UNIQUE NOT NULL,
    EMAIL  VARCHAR(60) UNIQUE NOT NULL,
    PWD    VARCHAR(60) NOT NULL,
    PRIMARY KEY (C_ID)
);
```

✅ Part 1: Generate a pandas DataFrame of Customers
This script generates realistic sample customer data (including FNAME, LNAME, PHONE, EMAIL, PWD) for each C_ID provided.

In [10]:
import pandas as pd
import random
import string
import json
from time import sleep
from tqdm import tqdm

In [11]:
# customer_data_generator.py




with open('names.json', 'r') as file:
    names = json.load(file)

def generate_phone():
    return '0'+ str(random.randint(1,9)) +''.join(random.choices(string.digits, k=8))

def generate_email(fname, lname):
    domains = ['example', 'test.com', 'demo.org']
    extensions = ['.com','.org', '.co']
    return f"{fname.lower()}.{lname.lower()}{random.randint(0,99)}@{random.choice(domains)}{random.choice(extensions)}"

def generate_password():
    return ''.join(random.choices(string.ascii_letters + string.digits, k=12))  # mock hash

def generate_customer_df(c_ids):
    fnames = names["first_names"]

    lnames = names["last_names"]

    data = []
    used_phones = set()
    used_emails = set()

    for c_id in tqdm(c_ids):
        fname = random.choice(fnames)
        lname = random.choice(lnames)

        phone = generate_phone()
        while phone in used_phones:
            phone = generate_phone()
        used_phones.add(phone)

        email = generate_email(fname, lname)
        while email in used_emails:
            email = generate_email(fname, lname)
        used_emails.add(email)

        pwd = generate_password()

        data.append({
            'c_id': c_id,
            'fname': fname,
            'lname': lname,
            'phone': phone,
            'email': email,
            'pwd': pwd
        })

    return pd.DataFrame(data)


✅ Part 2: Insert the DataFrame into PostgreSQL
Use this script to connect and insert the generated customers into your PostgreSQL CUSTOMER table.


In [59]:
# --- Generate DataFrame ---
# Ensure one default shipping address per customer

df_customer = generate_customer_df(c_ids)

100%|██████████| 100000/100000 [00:00<00:00, 235898.65it/s]


In [12]:
from sqlalchemy import create_engine

# --- CONFIGURATION ---
POSTGRES_URI = db_connection_string


In [ ]:

# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_customer.to_sql('customer', conn, if_exists='append', index=False)
    print(f"✅ Inserted {len(df_customer)} rows into 'customer'.")

In [15]:
# let's import the customer table from the db
import pandas as pd

engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_customer = pd.read_sql_table(table_name="customer",con=conn)

db_customer.head()

,c_id,fname,lname,phone,email,pwd
0,AE222BBOVZIF42YOOPNBXL4UUMYA,Connor,MASON,0604823672,connor.mason99@test.com.org,qOkVgJWdR9rk
1,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,Calvin,ANDERSON,0409711663,calvin.anderson6@example.com,uqt7PUOyH4TS
2,AE222X475JC6ONXMIKZDFGQ7IAUA,William,ANDREWS,0109386339,william.andrews27@test.com.org,4CHPsiqjjhh4
3,AE222Y4WTST6BUZ4J5Y2H6QMBITQ,Antonio,COLLINS,0778653405,antonio.collins80@example.org,AytJdxIZAaoA
4,AE2232TEZOEWQLAFEX2NA6VBGMYQ,Robert,MILLS,0526056936,robert.mills52@example.org,VNXPKkKnB1Os


### Shipping details

```sql
-- SHIPPING_DETAILS TABLE
CREATE TABLE SHIPPING_DETAILS (
    ADDRESS_ID INTEGER GENERATED BY DEFAULT AS IDENTITY,
    STREET_ADDRESS VARCHAR(100) NOT NULL,
    CITY VARCHAR(100) NOT NULL,
    STATE VARCHAR(100) NOT NULL,
    ZIP VARCHAR(10) NOT NULL,
    COUNTRY VARCHAR(60) NOT NULL,
    PHONE CHAR(10) NOT NULL,
    PRIMARY KEY (ADDRESS_ID)
);
```

In [62]:
# get a list of phone numbers from the customer table
# since all shipping address is linked to a phone number, might as well have all phone numbers being extracted
# from the customer table
import random
repeats = [random.randint(a=1,b=3) for i in range(len(db_customer["phone"]))] # each phone number will be repeated 1 to 3
# times, leading to each cutomer having 1 to 3 shipping addresses
customer_phone_numbers = random.sample(sorted(db_customer["phone"]),
                                       counts=repeats,
                                       k=sum(repeats))

In [63]:
from faker import Faker
import pandas as pd
from sqlalchemy import create_engine

# Initialize Faker
fake = Faker()

# Generate shipping details
def generate_shipping_details():
    data = []
    for phone in tqdm(customer_phone_numbers):
        data.append({
            "street_address": fake.street_address(),
            "city": fake.city(),
            "state": fake.state(),
            "zip": fake.postcode(),
            "country": "United States of America",
            "phone": phone  # Limit to 10 digits
        })
    return pd.DataFrame(data)

df_shipping_details = generate_shipping_details()

# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_shipping_details.to_sql('shipping_details', conn, if_exists='append', index=False)
    print(f"✅ Inserted {len(df_shipping_details)} rows into 'shipping_details'.")


100%|██████████| 200001/200001 [00:13<00:00, 14977.93it/s]


✅ Inserted 200001 rows into 'shipping_details'.


### Customer Shipping

```sql
-- CUSTOMER_SHIPPING TABLE
CREATE TABLE CUSTOMER_SHIPPING (
    ADDRESS_ID INTEGER,
    C_ID VARCHAR(40),
    IS_DEFAULT CHAR(1) DEFAULT '0' CHECK (IS_DEFAULT IN ('0', '1')),
    PRIMARY KEY (ADDRESS_ID, C_ID),
    CONSTRAINT CSADDRESSFK FOREIGN KEY (ADDRESS_ID) REFERENCES SHIPPING_DETAILS (ADDRESS_ID) ON DELETE CASCADE,
    CONSTRAINT CSCUSTOMERFK FOREIGN KEY (C_ID) REFERENCES CUSTOMER (C_ID) ON DELETE CASCADE
);
```

In [64]:
engine = create_engine(POSTGRES_URI)
with engine.begin() as conn:
    db_shipping_details = pd.read_sql_table(table_name="shipping_details",con=conn)

db_shipping_details.head()

,address_id,street_address,city,state,zip,country,phone
0,1,3255 Dixon Forks,New Linda,South Dakota,78712,United States of America,0624985265
1,2,341 Alyssa Loaf Apt. 931,North Ronaldmouth,Indiana,15870,United States of America,0252256632
2,3,171 Mccormick Inlet,Lake Bryan,Tennessee,45875,United States of America,0356087534
3,4,701 James Trail Apt. 508,Rileyfurt,Michigan,95318,United States of America,0690889448
4,5,8760 Linda Plains,Hoovermouth,Maryland,07188,United States of America,0734088352


In [65]:
default_address = db_shipping_details.drop_duplicates(subset="phone")[["phone","address_id"]]
default_address["is_default"] = "1"
default_address

,phone,address_id,is_default
0,0624985265,1,1
1,0252256632,2,1
2,0356087534,3,1
3,0690889448,4,1
4,0734088352,5,1
...,...,...,...
199969,0880249686,199970,1
199984,0151387737,199985,1
199990,0706978865,199991,1
199993,0382008788,199994,1


In [66]:
df_customer_shipping = pd.merge(left=db_customer,right=db_shipping_details,on="phone")
df_customer_shipping = pd.merge(left=df_customer_shipping,right=default_address,how="left",on=["phone","address_id"])
df_customer_shipping.fillna("0",inplace=True)
df_customer_shipping

,c_id,fname,lname,phone,email,pwd,address_id,street_address,city,state,zip,country,is_default
0,AE222BBOVZIF42YOOPNBXL4UUMYA,Connor,MASON,0604823672,connor.mason99@test.com.org,qOkVgJWdR9rk,36314,3056 Sara Parks Suite 511,Port Robinshire,New York,06963,United States of America,1
1,AE222BBOVZIF42YOOPNBXL4UUMYA,Connor,MASON,0604823672,connor.mason99@test.com.org,qOkVgJWdR9rk,55874,916 Williams Grove Apt. 962,Nixonfurt,Utah,64737,United States of America,0
2,AE222BBOVZIF42YOOPNBXL4UUMYA,Connor,MASON,0604823672,connor.mason99@test.com.org,qOkVgJWdR9rk,68068,45406 Sherry Cove Apt. 306,Sharpchester,Maine,60572,United States of America,0
3,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,Calvin,ANDERSON,0409711663,calvin.anderson6@example.com,uqt7PUOyH4TS,97108,548 Mendoza Forge,West Cynthia,Arizona,62999,United States of America,1
4,AE222X475JC6ONXMIKZDFGQ7IAUA,William,ANDREWS,0109386339,william.andrews27@test.com.org,4CHPsiqjjhh4,180015,218 Janet Mission Apt. 082,Gonzalezview,New York,18027,United States of America,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199996,AEO7LHP34RKZ4BZMFLUSCN3ZKDFQ,Tanner,ROGERS,0342195099,tanner.rogers80@demo.org.co,4VwEGBriYGd2,95091,6942 Medina Coves,Leonardside,New Hampshire,66579,United States of America,0
199997,AEO7LJ5DKX4CCINGZD3L74KP7SCQ,Cooper,ELLIS,0904650129,cooper.ellis53@example.org,SkNoNsbcOMW1,53631,64320 Nicholas Village Suite 981,North Melissa,Maine,58597,United States of America,1
199998,AEO7LX7MRZMII5QOEQT7UUR5AIKA,Eduardo,BAILEY,0873663538,eduardo.bailey85@test.com.com,sJuPI4g4QLxY,39886,389 Lisa Park Suite 758,New Haley,Mississippi,96668,United States of America,1
199999,AEO7LX7MRZMII5QOEQT7UUR5AIKA,Eduardo,BAILEY,0873663538,eduardo.bailey85@test.com.com,sJuPI4g4QLxY,72117,0435 Jackson Plaza Apt. 362,Port Mary,New Mexico,18913,United States of America,0


In [67]:
import pandas as pd
import random
from sqlalchemy import create_engine

# --- Generate DataFrame ---
# Ensure one default shipping address per customer

df_customer_shipping = df_customer_shipping[["c_id","address_id","is_default"]]

# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_customer_shipping.to_sql('customer_shipping', conn, if_exists='append', index=False)
    print(f"✅ Inserted {len(df_customer_shipping)} rows into 'customer_shipping'.")


✅ Inserted 200001 rows into 'customer_shipping'.


### PAYMENT_DETAILS

```sql
-- PAYMENT_DETAILS TABLE
CREATE TABLE PAYMENT_DETAILS (
    PAYMENT_ID INTEGER GENERATED BY DEFAULT AS IDENTITY,
    CARD_NO BIGINT NOT NULL,
    CVV SMALLINT NOT NULL,
    EXPIRY_DATE DATE NOT NULL,
    BILLING_ADDRESS VARCHAR(500) NOT NULL,
    PRIMARY KEY (PAYMENT_ID)
);
```

In [72]:
# FOR payment details we will use informations from the default address, and consider that all customer register 
# their payment method by associating it with their default address.
customer_shipping_default = df_customer_shipping.loc[df_customer_shipping["is_default"]=="1",:]
customer_shipping_default_complete = pd.merge(customer_shipping_default,
                                              db_shipping_details, on="address_id")
customer_shipping_default_complete.head()

,c_id,address_id,is_default,street_address,city,state,zip,country,phone
0,AE222BBOVZIF42YOOPNBXL4UUMYA,36314,1,3056 Sara Parks Suite 511,Port Robinshire,New York,06963,United States of America,0604823672
1,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,97108,1,548 Mendoza Forge,West Cynthia,Arizona,62999,United States of America,0409711663
2,AE222X475JC6ONXMIKZDFGQ7IAUA,180015,1,218 Janet Mission Apt. 082,Gonzalezview,New York,18027,United States of America,0109386339
3,AE222Y4WTST6BUZ4J5Y2H6QMBITQ,160400,1,31935 Chavez Mountains Apt. 665,Port Paulmouth,Wyoming,39290,United States of America,0778653405
4,AE2232TEZOEWQLAFEX2NA6VBGMYQ,8479,1,97336 Danielle Pass Suite 501,North Markfurt,Utah,89845,United States of America,0526056936


In [73]:
import pandas as pd
import random
from faker import Faker
from sqlalchemy import create_engine
from datetime import datetime, timedelta


# --- Generate DataFrame ---
fake = Faker()
payment_data = []

for i, row in customer_shipping_default_complete.iterrows():
    card_no = fake.credit_card_number()
    cvv = int(fake.credit_card_security_code())
    expiry_date = fake.date_between(start_date="+1y", end_date="+5y")
    billing_address = row["street_address"] + row["city"] +	row["state"] + row["zip"]
    c_id = row["c_id"]
    
    payment_data.append({
        "card_no": int(card_no),
        "cvv": cvv,
        "expiry_date": expiry_date,
        "billing_address": billing_address,
        "c_id": c_id
    })

df_payment = pd.DataFrame(payment_data)
df_payment_no_c_id = df_payment.drop("c_id", axis=1)



In [74]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_payment_no_c_id.to_sql("payment_details", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_payment)} rows into 'payment_details'.")


✅ Inserted 100000 rows into 'payment_details'.


### Payment customer

```sql
-- CUSTOMER_PAYMENT TABLE
CREATE TABLE CUSTOMER_PAYMENT (
    PAYMENT_ID INTEGER,
    C_ID VARCHAR(40),
    IS_DEFAULT CHAR(1) DEFAULT '0' CHECK (IS_DEFAULT IN ('0', '1')),
    PRIMARY KEY (C_ID, PAYMENT_ID),
    CONSTRAINT CPPAYMENTFK FOREIGN KEY (PAYMENT_ID) REFERENCES PAYMENT_DETAILS (PAYMENT_ID) ON DELETE CASCADE,
    CONSTRAINT CPCUSTOMERFK FOREIGN KEY (C_ID) REFERENCES CUSTOMER (C_ID) ON DELETE CASCADE
);
```

In [75]:
# --- Read from PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_payment_details = pd.read_sql_table("payment_details", con=conn)

In [76]:
db_payment_details.head()

,payment_id,card_no,cvv,expiry_date,billing_address
0,1,4419335152341009788,763,2027-04-18,3056 Sara Parks Suite 511Port RobinshireNew Yo...
1,2,5149307510258940,311,2029-09-06,548 Mendoza ForgeWest CynthiaArizona62999
2,3,6522716193634794,378,2027-01-26,218 Janet Mission Apt. 082GonzalezviewNew York...
3,4,4874202337238,383,2028-07-27,31935 Chavez Mountains Apt. 665Port PaulmouthW...
4,5,5493877233592579,1126,2029-06-18,97336 Danielle Pass Suite 501North MarkfurtUta...


In [77]:
df_customer_payment = pd.merge(df_payment,db_payment_details, on="billing_address")[["c_id","payment_id"]]
df_customer_payment["is_default"] = "1"
df_customer_payment

,c_id,payment_id,is_default
0,AE222BBOVZIF42YOOPNBXL4UUMYA,1,1
1,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,2,1
2,AE222X475JC6ONXMIKZDFGQ7IAUA,3,1
3,AE222Y4WTST6BUZ4J5Y2H6QMBITQ,4,1
4,AE2232TEZOEWQLAFEX2NA6VBGMYQ,5,1
...,...,...,...
99995,AEO7KTNXRQGBS2YUKRWB6HX3E3YQ,99996,1
99996,AEO7L5G33NXKAHIWJBNFGQCDJRUA,99997,1
99997,AEO7LHP34RKZ4BZMFLUSCN3ZKDFQ,99998,1
99998,AEO7LJ5DKX4CCINGZD3L74KP7SCQ,99999,1


In [80]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_customer_payment.to_sql("customer_payment", con=conn,if_exists="append",index=False)
    print(f"✅ Inserted {len(df_payment)} rows into 'payment_details'.")

✅ Inserted 100000 rows into 'payment_details'.


### Subscription

```sql
-- SUBSCRIPTION TABLE
CREATE TABLE SUBSCRIPTION (
    SUBSCRIPTION_ID  INTEGER
        GENERATED BY DEFAULT AS IDENTITY,
    C_ID             VARCHAR(40),
    START_DATE       DATE NOT NULL,
    END_DATE         DATE NOT NULL,
    PRIMARY KEY ( SUBSCRIPTION_ID ),
    CONSTRAINT SCUSTOMERFK FOREIGN KEY ( C_ID )
        REFERENCES CUSTOMER ( C_ID )
            ON DELETE SET NULL
);
```

In [82]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_customer = pd.read_sql_table("customer",con=conn)
db_customer.head()

,c_id,fname,lname,phone,email,pwd
0,AE222BBOVZIF42YOOPNBXL4UUMYA,Connor,MASON,0604823672,connor.mason99@test.com.org,qOkVgJWdR9rk
1,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,Calvin,ANDERSON,0409711663,calvin.anderson6@example.com,uqt7PUOyH4TS
2,AE222X475JC6ONXMIKZDFGQ7IAUA,William,ANDREWS,0109386339,william.andrews27@test.com.org,4CHPsiqjjhh4
3,AE222Y4WTST6BUZ4J5Y2H6QMBITQ,Antonio,COLLINS,0778653405,antonio.collins80@example.org,AytJdxIZAaoA
4,AE2232TEZOEWQLAFEX2NA6VBGMYQ,Robert,MILLS,0526056936,robert.mills52@example.org,VNXPKkKnB1Os


In [83]:
import pandas as pd
import random
from faker import Faker
from sqlalchemy import create_engine
from datetime import timedelta

# --- Generate Data ---
fake = Faker()
subscription_data = []

for c_id in tqdm(db_customer["c_id"]):
    if random.random() > 0.5:
        continue
    customer_id = c_id
    start_date = fake.date_between(start_date="-1y", end_date="today")
    end_date = start_date + timedelta(days=365)
    
    subscription_data.append({
        "c_id": customer_id,
        "start_date": start_date,
        "end_date": end_date
    })

df_subscription = pd.DataFrame(subscription_data)

# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_subscription.to_sql("subscription", conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_subscription)} rows into 'subscription'.")


100%|██████████| 100000/100000 [00:00<00:00, 219872.08it/s]


✅ Inserted 50091 rows into 'subscription'.


### BUYER

```sql
-- BUYER TABLE
CREATE TABLE BUYER (
    BUYER_ID VARCHAR(40),
    PRIMARY KEY ( BUYER_ID ),
    CONSTRAINT BBUYERFK FOREIGN KEY ( BUYER_ID )
        REFERENCES CUSTOMER ( C_ID )
            ON DELETE CASCADE
);
```

In [85]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_customer = pd.read_sql_table("customer",con=conn)
db_customer.head()

,c_id,fname,lname,phone,email,pwd
0,AE222BBOVZIF42YOOPNBXL4UUMYA,Connor,MASON,0604823672,connor.mason99@test.com.org,qOkVgJWdR9rk
1,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,Calvin,ANDERSON,0409711663,calvin.anderson6@example.com,uqt7PUOyH4TS
2,AE222X475JC6ONXMIKZDFGQ7IAUA,William,ANDREWS,0109386339,william.andrews27@test.com.org,4CHPsiqjjhh4
3,AE222Y4WTST6BUZ4J5Y2H6QMBITQ,Antonio,COLLINS,0778653405,antonio.collins80@example.org,AytJdxIZAaoA
4,AE2232TEZOEWQLAFEX2NA6VBGMYQ,Robert,MILLS,0526056936,robert.mills52@example.org,VNXPKkKnB1Os


In [86]:
df_buyer = db_customer["c_id"]
df_buyer.name = "buyer_id"
df_buyer.head()

0    AE222BBOVZIF42YOOPNBXL4UUMYA
1    AE222FP7YRNFCEQ2W3ZDIGMSYTLQ
2    AE222X475JC6ONXMIKZDFGQ7IAUA
3    AE222Y4WTST6BUZ4J5Y2H6QMBITQ
4    AE2232TEZOEWQLAFEX2NA6VBGMYQ
Name: buyer_id, dtype: object

In [87]:
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_buyer.to_sql("buyer",con=conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_buyer)} rows into 'buyer'.")


✅ Inserted 100000 rows into 'buyer'.


### Seller

```sql
-- SELLER TABLE
CREATE TABLE SELLER (
    SELLER_ID    VARCHAR(40),
    SELLER_TYPE  VARCHAR(200),
    SELLER_NAME VARCHAR(1000),
    PRIMARY KEY ( SELLER_ID ),
    CONSTRAINT SSELLERFK FOREIGN KEY ( SELLER_ID )
        REFERENCES CUSTOMER ( C_ID )
            ON DELETE CASCADE
);
```

In [20]:
df_metadata = pd.DataFrame(metadata)
df_metadata.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Howard Products,[],"{""Package Dimensions"": ""7.1 x 5.5 x 3 inches; ...",B01CUPMQZE,None,None,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Yes To,[],"{""Item Form"": ""Powder"", ""Skin Type"": ""Acne Pro...",B076WQZGPM,None,None,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],None,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Levine Health Products,[],"{""Manufacturer"": ""Levine Health Products""}",B000B658RI,None,None,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Cherioll,[],"{""Brand"": ""Cherioll"", ""Item Form"": ""Powder"", ""...",B088FKY3VD,None,None,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Precision,[],"{""UPC"": ""644287689178""}",B07NGFDN6G,None,None,None


In [21]:
# we are going to need to add sellers both to the customer and seller table

df_seller = df_metadata

df_seller_customer = df_seller

In [22]:
# --- Read from PostgreSQL ---
from sqlalchemy import create_engine

engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_customer = pd.read_sql_table("customer",con=conn)
db_customer.head()

,c_id,fname,lname,phone,email,pwd
0,AE222BBOVZIF42YOOPNBXL4UUMYA,Connor,MASON,0604823672,connor.mason99@test.com.org,qOkVgJWdR9rk
1,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,Calvin,ANDERSON,0409711663,calvin.anderson6@example.com,uqt7PUOyH4TS
2,AE222X475JC6ONXMIKZDFGQ7IAUA,William,ANDREWS,0109386339,william.andrews27@test.com.org,4CHPsiqjjhh4
3,AE222Y4WTST6BUZ4J5Y2H6QMBITQ,Antonio,COLLINS,0778653405,antonio.collins80@example.org,AytJdxIZAaoA
4,AE2232TEZOEWQLAFEX2NA6VBGMYQ,Robert,MILLS,0526056936,robert.mills52@example.org,VNXPKkKnB1Os


In [23]:
data = []
used_phones = set(db_customer["phone"])
used_emails = set(db_customer["email"])
used_ids = set(db_customer["c_id"])    

names = df_seller["store"].dropna().unique()

def generate_email_business(fname):
    extensions = ['.com','.org', '.co']
    return f"contact@{fname.lower()}{random.randint(0,99)}{random.choice(extensions)}".replace(" ","")


for name in tqdm(names):

    fname = name
    lname = "company"

    phone = generate_phone()
    while phone in used_phones:
        phone = generate_phone()
    used_phones.add(phone)

    email = generate_email_business(fname)
    while email in used_emails:
        email = generate_email_business(fname)
    used_emails.add(email)

    pwd = generate_password()

    c_id = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(30))
    while c_id in used_ids:
        c_id = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(30))
    used_ids.add(c_id)

    data.append({
        'c_id': c_id,
        'fname': fname,
        'lname': lname,
        'phone': phone,
        'email': email,
        'pwd': pwd
    })
df_seller_customer = pd.DataFrame(data)
df_seller_customer.head()

100%|██████████| 30765/30765 [00:00<00:00, 87007.37it/s]


,c_id,fname,lname,phone,email,pwd
0,D9B0PXSE72OGYLM7GK8NBFQRUB0JCQ,Howard Products,company,0203651402,contact@howardproducts84.co,9iK2xdowjNf6
1,17R4AR18XLD891TAJ6VNJPE6AXIR9F,Yes To,company,0935557541,contact@yesto16.co,nWrCOYld5OAl
2,Q8EF2UW1K7Z73ZNSH5HCRR4T888IQ3,Levine Health Products,company,0525220081,contact@levinehealthproducts20.org,7Dxh3oUWqDoP
3,QP1BCLEMT362APTP2K7AIHR7L36GRA,Cherioll,company,0785535274,contact@cherioll36.org,ws9jc1nqnys8
4,KY49WWAPULLJNJLDYTC2E6OB0MKRG3,Precision,company,0767136197,contact@precision62.co,2W0YI1j9fCLV


In [41]:
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_seller_customer.to_sql("customer",con=conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_seller_customer)} rows into 'customer'.")

✅ Inserted 30765 rows into 'customer'.


In [24]:
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_customer = pd.read_sql_table("customer", con=conn)
mask = db_customer["lname"]=="company"
db_customer_seller = db_customer.loc[mask,:]

In [25]:
db_customer_seller["seller_type"] = "third_party_seller"
db_customer_seller["seller_id"] = db_customer_seller["c_id"]
df_seller = db_customer_seller[["seller_id", "seller_type"]]

/var/folders/d4/h2gdkyfd23g119wvt_vqqv3h0000gn/T/ipykernel_2134/395957847.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_customer_seller["seller_type"] = "third_party_seller"
/var/folders/d4/h2gdkyfd23g119wvt_vqqv3h0000gn/T/ipykernel_2134/395957847.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_customer_seller["seller_id"] = db_customer_seller["c_id"]


In [26]:
df_seller.head()

,seller_id,seller_type
100001,UEZL3E2XAUHASTLXILVQZKMKC9P7MF,third_party_seller
100002,9GWGTCHNDFWCX3HN67WUF2JLO099JI,third_party_seller
100003,M98ZC7L20803D30SWJILXBJZ5DYHOG,third_party_seller
100004,C8TH52MDB77ZE0F83IJND85M8J7NPS,third_party_seller
100005,GUPY8BCNDLBY6CP5EU479AY4NXEG1O,third_party_seller


In [27]:
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_seller.to_sql("seller",con=conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_seller)} rows into 'seller'.")

✅ Inserted 30765 rows into 'seller'.


In [28]:
# SQL statements to run (e.g. CREATE TABLE)
SCHEMA_QUERIES = [
    """
-- 1) Insert Amazon as a customer
INSERT INTO CUSTOMER (
    C_ID,
    FNAME,
    LNAME,
    PHONE,
    EMAIL,
    PWD
) VALUES (
    'AMAZON0000000000000000001',  -- 30-char ID
    'Amazon',
    'Inc',
    '0000000000',                 -- dummy 10-digit phone
    'contact@amazon.com',
    'secure_password_hash'        -- store an actual hash in production
);""",
"""

-- 2) Insert Amazon into the seller table
INSERT INTO SELLER (
    SELLER_ID,
    SELLER_TYPE
) VALUES (
    'AMAZON0000000000000000001',  -- must match CUSTOMER.C_ID
    'marketplace'
);    """,
    # … add more DDL here …
]

def run_schema_queries():
    conn = None
    try:
        # Connect
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()

        # Execute all queries
        for ddl in SCHEMA_QUERIES[1:]:
            cur.execute(ddl)
            print("✅ Executed:\n", ddl.strip(), "\n")

        # Persist
        conn.commit()
        print("All records inserted successfully.")

    except Exception as e:
        print("❌ Error:", e)

    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    run_schema_queries()


✅ Executed:
 -- 2) Insert Amazon into the seller table
INSERT INTO SELLER (
    SELLER_ID,
    SELLER_TYPE
) VALUES (
    'AMAZON0000000000000000001',  -- must match CUSTOMER.C_ID
    'marketplace'
); 

All records inserted successfully.


### Review

```sql
-- REVIEW TABLE
CREATE TABLE REVIEW (
    REVIEW_ID            INTEGER
        GENERATED BY DEFAULT AS IDENTITY,
    BUYER_ID             VARCHAR(40),
    R_DESC               VARCHAR(20000) NOT NULL,
    TITLE                VARCHAR(300) NOT NULL,
    RATING               INT CHECK ( RATING BETWEEN 1 AND 5 ),
    SELLER_PRODUCT_FLAG  CHAR(1) CHECK ( SELLER_PRODUCT_FLAG IN ( 'S', 'P' ) ),
    PRIMARY KEY ( REVIEW_ID ),
    CONSTRAINT RBUYERFK FOREIGN KEY ( BUYER_ID )
        REFERENCES BUYER ( BUYER_ID )
            ON DELETE SET NULL
);
```
we are going to extract all reviews associated with customers from the buyer table

In [63]:
# --- Read from PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_buyer = pd.read_sql_table("buyer",con=conn)
db_buyer

,buyer_id
0,AE222BBOVZIF42YOOPNBXL4UUMYA
1,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ
2,AE222X475JC6ONXMIKZDFGQ7IAUA
3,AE222Y4WTST6BUZ4J5Y2H6QMBITQ
4,AE2232TEZOEWQLAFEX2NA6VBGMYQ
...,...
99995,AEO7KTNXRQGBS2YUKRWB6HX3E3YQ
99996,AEO7L5G33NXKAHIWJBNFGQCDJRUA
99997,AEO7LHP34RKZ4BZMFLUSCN3ZKDFQ
99998,AEO7LJ5DKX4CCINGZD3L74KP7SCQ


In [33]:
buyers_list = db_buyer["buyer_id"].apply(lambda x: x.strip()).to_list()
review = dataset["full"].filter(lambda x: x["user_id"] in buyers_list)

In [65]:
df_review = pd.DataFrame(review)
len(df_review)

111322

In [103]:
df_review = df_review.rename({"text":"r_desc",
                  "user_id":"buyer_id"},
                  axis=1)
df_review = df_review[["buyer_id", "r_desc", "title", "rating"]]
df_review["seller_product_flag"] = "S"
df_review.head()

,buyer_id,r_desc,title,rating,seller_product_flag
0,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,"Smells good, feels great!",Yes!,5.0,S
1,AE5DIA2HDWBPNGBO2FXN2PF4NQJA,These are not what I was expecting. They are s...,Really nice quality,5.0,S
2,AEEM2QIBMUUXEAJIEQH2WT2KNZZA,Great product,Great product,5.0,S
3,AEITH2MBDGMCSAZSBNAYOEBOJXFA,only 2 problems with this product--expensive a...,great clipper cleaner,5.0,S
4,AEHVSZ72XXI4SU6XUNXMJAXUYN6Q,I'm very happy with my order. Will order item ...,Four Stars,4.0,S


In [106]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_review.to_sql("review", con=conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_review)} rows into 'review'.")

✅ Inserted 111322 rows into 'review'.


### Seller reviews

```sql
-- SELLER_REVIEWS TABLE
CREATE TABLE SELLER_REVIEWS (
    SELLER_ID  VARCHAR(40),
    REVIEW_ID  INTEGER,
    PRIMARY KEY ( SELLER_ID,
                  REVIEW_ID ),
    CONSTRAINT SRSELLERFK FOREIGN KEY ( SELLER_ID )
        REFERENCES SELLER ( SELLER_ID )
            ON DELETE CASCADE,
    CONSTRAINT SRREVIEWFK FOREIGN KEY ( REVIEW_ID )
        REFERENCES REVIEW ( REVIEW_ID )
            ON DELETE CASCADE
);
```

In [29]:
# --- Read from PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_review = pd.read_sql("review", con=conn)
    db_seller = pd.read_sql("seller", con=conn)
    db_customer = pd.read_sql_table("customer", con=conn)
db_review.head()

,review_id,buyer_id,r_desc,title,rating,seller_product_flag
0,96001,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,"Smells good, feels great!",Yes!,5,S
1,96002,AE5DIA2HDWBPNGBO2FXN2PF4NQJA,These are not what I was expecting. They are s...,Really nice quality,5,S
2,96003,AEEM2QIBMUUXEAJIEQH2WT2KNZZA,Great product,Great product,5,S
3,96004,AEITH2MBDGMCSAZSBNAYOEBOJXFA,only 2 problems with this product--expensive a...,great clipper cleaner,5,S
4,96005,AEHVSZ72XXI4SU6XUNXMJAXUYN6Q,I'm very happy with my order. Will order item ...,Four Stars,4,S


In [30]:
db_seller.head()

,seller_id,seller_type
0,UEZL3E2XAUHASTLXILVQZKMKC9P7MF,third_party_seller
1,9GWGTCHNDFWCX3HN67WUF2JLO099JI,third_party_seller
2,M98ZC7L20803D30SWJILXBJZ5DYHOG,third_party_seller
3,C8TH52MDB77ZE0F83IJND85M8J7NPS,third_party_seller
4,GUPY8BCNDLBY6CP5EU479AY4NXEG1O,third_party_seller


In [31]:
db_customer.head()

,c_id,fname,lname,phone,email,pwd
0,AE222BBOVZIF42YOOPNBXL4UUMYA,Connor,MASON,0604823672,connor.mason99@test.com.org,qOkVgJWdR9rk
1,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,Calvin,ANDERSON,0409711663,calvin.anderson6@example.com,uqt7PUOyH4TS
2,AE222X475JC6ONXMIKZDFGQ7IAUA,William,ANDREWS,0109386339,william.andrews27@test.com.org,4CHPsiqjjhh4
3,AE222Y4WTST6BUZ4J5Y2H6QMBITQ,Antonio,COLLINS,0778653405,antonio.collins80@example.org,AytJdxIZAaoA
4,AE2232TEZOEWQLAFEX2NA6VBGMYQ,Robert,MILLS,0526056936,robert.mills52@example.org,VNXPKkKnB1Os


In [34]:
df_review = pd.DataFrame(review)
df_review.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
1,5.0,Really nice quality,These are not what I was expecting. They are s...,[],B081GCFHPG,B081GCFHPG,AE5DIA2HDWBPNGBO2FXN2PF4NQJA,1623459734108,0,True
2,5.0,Great product,Great product,[],B07T2L6JQR,B07T2L6JQR,AEEM2QIBMUUXEAJIEQH2WT2KNZZA,1656871648888,0,True
3,5.0,great clipper cleaner,only 2 problems with this product--expensive a...,[],B0008F6QGO,B0008F6QGO,AEITH2MBDGMCSAZSBNAYOEBOJXFA,1402260693000,1,True
4,4.0,Four Stars,I'm very happy with my order. Will order item ...,[],B00JM1BZUW,B00JM1BZUW,AEHVSZ72XXI4SU6XUNXMJAXUYN6Q,1443695835000,0,True


In [36]:
df_metadata = pd.DataFrame(metadata)
df_metadata.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Howard Products,[],"{""Package Dimensions"": ""7.1 x 5.5 x 3 inches; ...",B01CUPMQZE,None,None,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Yes To,[],"{""Item Form"": ""Powder"", ""Skin Type"": ""Acne Pro...",B076WQZGPM,None,None,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],None,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Levine Health Products,[],"{""Manufacturer"": ""Levine Health Products""}",B000B658RI,None,None,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Cherioll,[],"{""Brand"": ""Cherioll"", ""Item Form"": ""Powder"", ""...",B088FKY3VD,None,None,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Precision,[],"{""UPC"": ""644287689178""}",B07NGFDN6G,None,None,None


In [46]:
df_metadata["store"].fillna("Amazon", inplace=True)

In [47]:
df_metadata.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Howard Products,[],"{""Package Dimensions"": ""7.1 x 5.5 x 3 inches; ...",B01CUPMQZE,None,None,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Yes To,[],"{""Item Form"": ""Powder"", ""Skin Type"": ""Acne Pro...",B076WQZGPM,None,None,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],None,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Levine Health Products,[],"{""Manufacturer"": ""Levine Health Products""}",B000B658RI,None,None,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Cherioll,[],"{""Brand"": ""Cherioll"", ""Item Form"": ""Powder"", ""...",B088FKY3VD,None,None,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Precision,[],"{""UPC"": ""644287689178""}",B07NGFDN6G,None,None,None


In [48]:
df_seller_reviews = pd.merge(db_review,df_review, left_on="r_desc", right_on="text")
df_seller_reviews = pd.merge(df_seller_reviews, df_metadata, on="parent_asin")
df_seller_reviews = pd.merge(df_seller_reviews, db_customer, left_on="store", right_on="fname")
df_seller_reviews = pd.merge(df_seller_reviews, db_seller, left_on="c_id", right_on="seller_id")
df_seller_reviews = df_seller_reviews[["seller_id", "review_id"]].drop_duplicates("review_id")
                             
df_seller_reviews.head()

,seller_id,review_id
0,KVSMGEV7R4DXPP23UWP71GKH499BYE,96001
1,V7OKB7ZLGQSWH2TF77E2MHQLKEYGBF,96002
2,AMAZON0000000000000000001,96003
204,ZFMG7E95IPVQO5DEBEW27O71Y5P7JC,96004
205,AMAZON0000000000000000001,96005


In [49]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_seller_reviews.to_sql("seller_reviews", con=conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_seller_reviews)} rows into 'seller_reviews'.")

✅ Inserted 111322 rows into 'seller_reviews'.


### Review images

```sql
-- REVIEW_IMAGES TABLE
CREATE TABLE REVIEW_IMAGES (
    REVIEW_ID   INTEGER,
    REVIEW_IMG  VARCHAR(200),
    PRIMARY KEY ( REVIEW_ID,
                  REVIEW_IMG ),
    CONSTRAINT RIREVIEWFK FOREIGN KEY ( REVIEW_ID )
        REFERENCES REVIEW ( REVIEW_ID )
            ON DELETE CASCADE
);
```

In [158]:
df_review = pd.DataFrame(review)

In [159]:
df_review.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
1,5.0,Really nice quality,These are not what I was expecting. They are s...,[],B081GCFHPG,B081GCFHPG,AE5DIA2HDWBPNGBO2FXN2PF4NQJA,1623459734108,0,True
2,5.0,Great product,Great product,[],B07T2L6JQR,B07T2L6JQR,AEEM2QIBMUUXEAJIEQH2WT2KNZZA,1656871648888,0,True
3,5.0,great clipper cleaner,only 2 problems with this product--expensive a...,[],B0008F6QGO,B0008F6QGO,AEITH2MBDGMCSAZSBNAYOEBOJXFA,1402260693000,1,True
4,4.0,Four Stars,I'm very happy with my order. Will order item ...,[],B00JM1BZUW,B00JM1BZUW,AEHVSZ72XXI4SU6XUNXMJAXUYN6Q,1443695835000,0,True


In [160]:
# --- Read from PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_review = pd.read_sql("review", con=conn)
db_review.head()

,review_id,buyer_id,r_desc,title,rating,seller_product_flag
0,96001,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,"Smells good, feels great!",Yes!,5,S
1,96002,AE5DIA2HDWBPNGBO2FXN2PF4NQJA,These are not what I was expecting. They are s...,Really nice quality,5,S
2,96003,AEEM2QIBMUUXEAJIEQH2WT2KNZZA,Great product,Great product,5,S
3,96004,AEITH2MBDGMCSAZSBNAYOEBOJXFA,only 2 problems with this product--expensive a...,great clipper cleaner,5,S
4,96005,AEHVSZ72XXI4SU6XUNXMJAXUYN6Q,I'm very happy with my order. Will order item ...,Four Stars,4,S


In [183]:
df_review_image = pd.merge(db_review,df_review,left_on="r_desc", right_on="text")
df_review_image = df_review_image[["review_id", "images"]]
df_review_image = df_review_image.explode("images").dropna()
df_review_image["review_img"] = df_review_image['images'].apply(lambda x: [x["large_image_url"],
                                 x["medium_image_url"],
                                 x["small_image_url"]])
df_review_image = df_review_image.explode("review_img").dropna()[["review_id","review_img"]].drop_duplicates()
df_review_image

,review_id,review_img
200,96012,https://images-na.ssl-images-amazon.com/images...
200,96012,https://images-na.ssl-images-amazon.com/images...
200,96012,https://images-na.ssl-images-amazon.com/images...
200,96012,https://images-na.ssl-images-amazon.com/images...
200,96012,https://images-na.ssl-images-amazon.com/images...
...,...,...
461154,207321,https://images-na.ssl-images-amazon.com/images...
461154,207321,https://images-na.ssl-images-amazon.com/images...
461155,207322,https://images-na.ssl-images-amazon.com/images...
461155,207322,https://images-na.ssl-images-amazon.com/images...


In [184]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_review_image.to_sql("review_images", con=conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_review_image)} rows into 'review_image'.")

✅ Inserted 119382 rows into 'review_image'.


### Category

```sql
-- CATEGORY TABLE
CREATE TABLE CATEGORY (
    CATEGORY_ID  INTEGER
        GENERATED BY DEFAULT AS IDENTITY,
    NAME         VARCHAR(30) NOT NULL,
    C_DESC       VARCHAR(20) NOT NULL,
    PRIMARY KEY ( CATEGORY_ID )
);
```

In [185]:
df_metadata.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Howard Products,[],"{""Package Dimensions"": ""7.1 x 5.5 x 3 inches; ...",B01CUPMQZE,None,None,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Yes To,[],"{""Item Form"": ""Powder"", ""Skin Type"": ""Acne Pro...",B076WQZGPM,None,None,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],None,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Levine Health Products,[],"{""Manufacturer"": ""Levine Health Products""}",B000B658RI,None,None,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Cherioll,[],"{""Brand"": ""Cherioll"", ""Item Form"": ""Powder"", ""...",B088FKY3VD,None,None,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Precision,[],"{""UPC"": ""644287689178""}",B07NGFDN6G,None,None,None


In [189]:
df_category = df_metadata
df_category["name"] = df_category["main_category"]
df_category["c_desc"] = ""
df_category = df_category[["name", "c_desc"]].drop_duplicates()
df_category.loc[0,"c_desc"] = "beauty general"
df_category.loc[33,"c_desc"] = "beauty premium"
df_category



,name,c_desc
0,All Beauty,beauty general
33,Premium Beauty,beauty premium


In [190]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_category.to_sql("category", con=conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_category)} rows into 'category'.")

✅ Inserted 2 rows into 'category'.


### Product

```sql
-- PRODUCT TABLE
CREATE TABLE PRODUCT (
    P_ID         VARCHAR(10) NOT NULL,
    P_NAME       VARCHAR(1000) NOT NULL,
    P_DESC       VARCHAR(5000) NOT NULL,
    PRICE        NUMERIC NOT NULL,
    QTY          INTEGER DEFAULT 0 NOT NULL CHECK ( QTY >= 0 ),
    CATEGORY_ID  INTEGER,
    PRIMARY KEY ( P_ID ),
    CONSTRAINT PCATEGORYFK FOREIGN KEY ( CATEGORY_ID )
        REFERENCES CATEGORY ( CATEGORY_ID )
            ON DELETE SET NULL
);
```

In [192]:
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_category = pd.read_sql("category", con=conn)
db_category.head()

,category_id,name,c_desc
0,1,All Beauty,beauty general
1,2,Premium Beauty,beauty premium


In [115]:
df_review = pd.DataFrame(review)
df_review.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
1,5.0,Really nice quality,These are not what I was expecting. They are s...,[],B081GCFHPG,B081GCFHPG,AE5DIA2HDWBPNGBO2FXN2PF4NQJA,1623459734108,0,True
2,5.0,Great product,Great product,[],B07T2L6JQR,B07T2L6JQR,AEEM2QIBMUUXEAJIEQH2WT2KNZZA,1656871648888,0,True
3,5.0,great clipper cleaner,only 2 problems with this product--expensive a...,[],B0008F6QGO,B0008F6QGO,AEITH2MBDGMCSAZSBNAYOEBOJXFA,1402260693000,1,True
4,4.0,Four Stars,I'm very happy with my order. Will order item ...,[],B00JM1BZUW,B00JM1BZUW,AEHVSZ72XXI4SU6XUNXMJAXUYN6Q,1443695835000,0,True


In [117]:
df_metadata = pd.DataFrame(metadata)
df_metadata.head()


,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Howard Products,[],"{""Package Dimensions"": ""7.1 x 5.5 x 3 inches; ...",B01CUPMQZE,None,None,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Yes To,[],"{""Item Form"": ""Powder"", ""Skin Type"": ""Acne Pro...",B076WQZGPM,None,None,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],None,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Levine Health Products,[],"{""Manufacturer"": ""Levine Health Products""}",B000B658RI,None,None,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Cherioll,[],"{""Brand"": ""Cherioll"", ""Item Form"": ""Powder"", ""...",B088FKY3VD,None,None,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Precision,[],"{""UPC"": ""644287689178""}",B07NGFDN6G,None,None,None


In [193]:
df_product = pd.merge(df_review,df_metadata,how="left",on="parent_asin")
df_product = pd.merge(df_product,db_category, left_on="main_category", right_on="name")
df_product = df_product.rename({"parent_asin":"p_id",
                          "title_y":"p_name",
                          "details":"p_desc"}, axis=1)
df_product = df_product[["p_id", "p_name", "p_desc", "price","category_id"]]
df_product["qty"] = [random.randint(1,100) for _ in range (len(df_product))]
df_product.head()

,p_id,p_name,p_desc,price,category_id,qty
0,B097R46CSY,New Road Beauty - Creamsicle - Variety 3 Pack ...,"{""Package Dimensions"": ""10.5 x 6.4 x 1.6 inche...",21.98,1,84
1,B081GCFHPG,Bow Hair Band Lovely Rabbit Ears Soft Carol Fl...,"{""Material"": ""Microfiber"", ""Brand"": ""Elibelle""...",None,1,67
2,B07T2L6JQR,Tea Tree Lemon Sage Thickening Liter Duo Set,"{""Product Benefits"": ""Thickening agents boost ...",85.0,2,46
3,B0008F6QGO,H42 Clipper Cleaner 16 Oz. Jar Virucidal Anti-...,"{""Brand"": ""H-42"", ""Item Form"": ""Liquid"", ""Item...",16.5,1,74
4,B00JM1BZUW,Sannysis(TM) 1PC Useful Storage Case Box Holde...,{},None,1,50


In [194]:
df_product.loc[df_product["price"]=="None","price"] = np.nan
df_product["price"] = df_product["price"].astype("float")
mu_price = df_product["price"].mean()
sigma_price = df_product["price"].std()
df_product.loc[df_product["price"].isna(),"price"] = [abs(random.normalvariate(mu=mu_price,
                                                                     sigma=sigma_price))
                                                                     for _ in range(sum(df_product["price"].isna()))]
df_product = df_product.drop_duplicates("p_id")
df_product.head()

,p_id,p_name,p_desc,price,category_id,qty
0,B097R46CSY,New Road Beauty - Creamsicle - Variety 3 Pack ...,"{""Package Dimensions"": ""10.5 x 6.4 x 1.6 inche...",21.980000,1,84
1,B081GCFHPG,Bow Hair Band Lovely Rabbit Ears Soft Carol Fl...,"{""Material"": ""Microfiber"", ""Brand"": ""Elibelle""...",64.857700,1,67
2,B07T2L6JQR,Tea Tree Lemon Sage Thickening Liter Duo Set,"{""Product Benefits"": ""Thickening agents boost ...",85.000000,2,46
3,B0008F6QGO,H42 Clipper Cleaner 16 Oz. Jar Virucidal Anti-...,"{""Brand"": ""H-42"", ""Item Form"": ""Liquid"", ""Item...",16.500000,1,74
4,B00JM1BZUW,Sannysis(TM) 1PC Useful Storage Case Box Holde...,{},13.944515,1,50


In [195]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_product.to_sql("product", con=conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_product)} rows into 'product'.")

✅ Inserted 42858 rows into 'product'.


### Whishlist Item

```sql
-- WISHLIST_ITEM TABLE
CREATE TABLE WISHLIST_ITEM (
    PRODUCT_ID  VARCHAR(10),
    BUYER_ID    VARCHAR(40),
    PRIMARY KEY ( PRODUCT_ID,
                  BUYER_ID ),
    CONSTRAINT WIPRODUCTFK FOREIGN KEY ( PRODUCT_ID )
        REFERENCES PRODUCT ( P_ID )
            ON DELETE CASCADE,
    CONSTRAINT WIBUYERFK FOREIGN KEY ( BUYER_ID )
        REFERENCES BUYER ( BUYER_ID )
            ON DELETE CASCADE
);
```

In [13]:
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_product = pd.read_sql_table("product", con=conn)
    db_buyer = pd.read_sql_table("buyer", con=conn)

In [15]:
import random
import pandas as pd
import numpy as np

# Example input lists (replace these with your actual data)
product_ids = db_product["p_id"]
buyer_ids = db_buyer["buyer_id"]

# Generate wishlist entries
wishlist_records = []

for b_id in tqdm(buyer_ids):
    num_items = np.random.geometric(p=0.8)-1
    # Avoid sampling more items than available
    sampled_products = random.sample(product_ids.to_list(), min(num_items, len(product_ids)))
    for prod in sampled_products:
        wishlist_records.append({
            'product_id': prod,
            'buyer_id': b_id
        })

# Create a DataFrame
df_wishlist = pd.DataFrame(wishlist_records)

# Display the first few rows
df_wishlist.head()


100%|██████████| 100000/100000 [00:13<00:00, 7482.08it/s]


,product_id,buyer_id
0,B087J1T1H8,AE222Y4WTST6BUZ4J5Y2H6QMBITQ
1,B07TF9Q7SW,AE222Y4WTST6BUZ4J5Y2H6QMBITQ
2,B0109OU0QU,AE22ARLVNG2NGMGQYSUEUU6N3BOQ
3,B08D7NGJY8,AE22BJCLGXGMOFCGOUPWWIZH2Y6Q
4,B00WCQZKFA,AE22BJCLGXGMOFCGOUPWWIZH2Y6Q


In [ ]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_wishlist.to_sql("wishlist_item", con=conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_wishlist)} rows into 'whishlist item'.")

### Seller product

```sql
-- SELLER_PRODUCTS TABLE
CREATE TABLE SELLER_PRODUCTS (
    SELLER_ID  VARCHAR(40),
    P_ID       VARCHAR(10),
    PRIMARY KEY ( SELLER_ID,
                  P_ID ),
    CONSTRAINT SPSELLERFK FOREIGN KEY ( SELLER_ID )
        REFERENCES SELLER ( SELLER_ID )
            ON DELETE CASCADE,
    CONSTRAINT SPPRODUCTFK FOREIGN KEY ( P_ID )
        REFERENCES PRODUCT ( P_ID )
            ON DELETE CASCADE
);
```

In [50]:
# --- Read from PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_review = pd.read_sql("review", con=conn)
    db_seller = pd.read_sql("seller", con=conn)
    db_customer = pd.read_sql_table("customer", con=conn)
db_review.head()

,review_id,buyer_id,r_desc,title,rating,seller_product_flag
0,96001,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,"Smells good, feels great!",Yes!,5,S
1,96002,AE5DIA2HDWBPNGBO2FXN2PF4NQJA,These are not what I was expecting. They are s...,Really nice quality,5,S
2,96003,AEEM2QIBMUUXEAJIEQH2WT2KNZZA,Great product,Great product,5,S
3,96004,AEITH2MBDGMCSAZSBNAYOEBOJXFA,only 2 problems with this product--expensive a...,great clipper cleaner,5,S
4,96005,AEHVSZ72XXI4SU6XUNXMJAXUYN6Q,I'm very happy with my order. Will order item ...,Four Stars,4,S


In [51]:
db_seller.head()

,seller_id,seller_type
0,UEZL3E2XAUHASTLXILVQZKMKC9P7MF,third_party_seller
1,9GWGTCHNDFWCX3HN67WUF2JLO099JI,third_party_seller
2,M98ZC7L20803D30SWJILXBJZ5DYHOG,third_party_seller
3,C8TH52MDB77ZE0F83IJND85M8J7NPS,third_party_seller
4,GUPY8BCNDLBY6CP5EU479AY4NXEG1O,third_party_seller


In [52]:
db_customer.head()

,c_id,fname,lname,phone,email,pwd
0,AE222BBOVZIF42YOOPNBXL4UUMYA,Connor,MASON,0604823672,connor.mason99@test.com.org,qOkVgJWdR9rk
1,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,Calvin,ANDERSON,0409711663,calvin.anderson6@example.com,uqt7PUOyH4TS
2,AE222X475JC6ONXMIKZDFGQ7IAUA,William,ANDREWS,0109386339,william.andrews27@test.com.org,4CHPsiqjjhh4
3,AE222Y4WTST6BUZ4J5Y2H6QMBITQ,Antonio,COLLINS,0778653405,antonio.collins80@example.org,AytJdxIZAaoA
4,AE2232TEZOEWQLAFEX2NA6VBGMYQ,Robert,MILLS,0526056936,robert.mills52@example.org,VNXPKkKnB1Os


In [53]:
df_review = pd.DataFrame(review)
df_review.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
1,5.0,Really nice quality,These are not what I was expecting. They are s...,[],B081GCFHPG,B081GCFHPG,AE5DIA2HDWBPNGBO2FXN2PF4NQJA,1623459734108,0,True
2,5.0,Great product,Great product,[],B07T2L6JQR,B07T2L6JQR,AEEM2QIBMUUXEAJIEQH2WT2KNZZA,1656871648888,0,True
3,5.0,great clipper cleaner,only 2 problems with this product--expensive a...,[],B0008F6QGO,B0008F6QGO,AEITH2MBDGMCSAZSBNAYOEBOJXFA,1402260693000,1,True
4,4.0,Four Stars,I'm very happy with my order. Will order item ...,[],B00JM1BZUW,B00JM1BZUW,AEHVSZ72XXI4SU6XUNXMJAXUYN6Q,1443695835000,0,True


In [54]:
df_metadata = pd.DataFrame(metadata)
df_metadata["store"].fillna("Amazon", inplace=True)
df_metadata.head()

/var/folders/d4/h2gdkyfd23g119wvt_vqqv3h0000gn/T/ipykernel_2134/3561490255.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_metadata["store"].fillna("Amazon", inplace=True)


,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Howard Products,[],"{""Package Dimensions"": ""7.1 x 5.5 x 3 inches; ...",B01CUPMQZE,None,None,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Yes To,[],"{""Item Form"": ""Powder"", ""Skin Type"": ""Acne Pro...",B076WQZGPM,None,None,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],None,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Levine Health Products,[],"{""Manufacturer"": ""Levine Health Products""}",B000B658RI,None,None,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Cherioll,[],"{""Brand"": ""Cherioll"", ""Item Form"": ""Powder"", ""...",B088FKY3VD,None,None,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Precision,[],"{""UPC"": ""644287689178""}",B07NGFDN6G,None,None,None


In [83]:
df_seller_products = pd.merge(db_review,df_review, left_on="r_desc", right_on="text")
df_seller_products = pd.merge(df_seller_products, df_metadata, on="parent_asin")
df_seller_products = pd.merge(df_seller_products, db_customer, left_on="store", right_on="fname")
df_seller_products = pd.merge(df_seller_products, db_seller, left_on="c_id", right_on="seller_id")
df_seller_products["p_id"]= df_seller_products["parent_asin"]
df_seller_products = df_seller_products[["seller_id","p_id"]].drop_duplicates()

df_seller_products.head()

,seller_id,p_id
0,KVSMGEV7R4DXPP23UWP71GKH499BYE,B097R46CSY
1,V7OKB7ZLGQSWH2TF77E2MHQLKEYGBF,B081GCFHPG
2,AMAZON0000000000000000001,B07T2L6JQR
3,FFLB2TALJNM3XMTCJTJMFOJ8IKU3AJ,B07T2L6JQR
522,ZFMG7E95IPVQO5DEBEW27O71Y5P7JC,B0008F6QGO


In [ ]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_seller_products.to_sql("seller_products", con=conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_seller_products)} rows into 'seller products'.")

### Product reviews

```sql
-- PRODUCT_REVIEWS TABLE
CREATE TABLE PRODUCT_REVIEWS (
    P_ID       VARCHAR(10),
    REVIEW_ID  INTEGER,
    PRIMARY KEY ( P_ID,
                  REVIEW_ID ),
    CONSTRAINT PRPRODUCTFK FOREIGN KEY ( P_ID )
        REFERENCES PRODUCT ( P_ID )
            ON DELETE CASCADE,
    CONSTRAINT PRREVIEWFK FOREIGN KEY ( REVIEW_ID )
        REFERENCES REVIEW ( REVIEW_ID )
            ON DELETE CASCADE
);
```

In [60]:
# --- get from db PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_review = pd.read_sql_table("review", con=conn)

In [61]:
db_review.head()

,review_id,buyer_id,r_desc,title,rating,seller_product_flag
0,96001,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,"Smells good, feels great!",Yes!,5,S
1,96002,AE5DIA2HDWBPNGBO2FXN2PF4NQJA,These are not what I was expecting. They are s...,Really nice quality,5,S
2,96003,AEEM2QIBMUUXEAJIEQH2WT2KNZZA,Great product,Great product,5,S
3,96004,AEITH2MBDGMCSAZSBNAYOEBOJXFA,only 2 problems with this product--expensive a...,great clipper cleaner,5,S
4,96005,AEHVSZ72XXI4SU6XUNXMJAXUYN6Q,I'm very happy with my order. Will order item ...,Four Stars,4,S


In [62]:
df_review = pd.DataFrame(review)
df_review.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
1,5.0,Really nice quality,These are not what I was expecting. They are s...,[],B081GCFHPG,B081GCFHPG,AE5DIA2HDWBPNGBO2FXN2PF4NQJA,1623459734108,0,True
2,5.0,Great product,Great product,[],B07T2L6JQR,B07T2L6JQR,AEEM2QIBMUUXEAJIEQH2WT2KNZZA,1656871648888,0,True
3,5.0,great clipper cleaner,only 2 problems with this product--expensive a...,[],B0008F6QGO,B0008F6QGO,AEITH2MBDGMCSAZSBNAYOEBOJXFA,1402260693000,1,True
4,4.0,Four Stars,I'm very happy with my order. Will order item ...,[],B00JM1BZUW,B00JM1BZUW,AEHVSZ72XXI4SU6XUNXMJAXUYN6Q,1443695835000,0,True


In [63]:
df_product_reviews = pd.merge(df_review, db_review, left_on="text", right_on="r_desc")
df_product_reviews = df_product_reviews.drop_duplicates("review_id")
df_product_reviews = df_product_reviews.rename({"parent_asin":"p_id"}, axis=1)
df_product_reviews = df_product_reviews[["p_id","review_id"]]
df_product_reviews.head()

,p_id,review_id
0,B097R46CSY,96001
1,B081GCFHPG,96002
2,B07T2L6JQR,96003
3,B07T2L6JQR,96109
4,B07T2L6JQR,96842


In [64]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_product_reviews.to_sql("product_reviews", con=conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_product_reviews)} rows into 'product reviews'.")

✅ Inserted 111322 rows into 'product reviews'.


### Product images

```sql
-- PRODUCT_IMAGES TABLE
CREATE TABLE PRODUCT_IMAGES (
    P_ID     VARCHAR(10),
    P_IMAGE  CHAR(20),
    PRIMARY KEY ( P_ID,
                  P_IMAGE ),
    CONSTRAINT PIPRODUCTFK FOREIGN KEY ( P_ID )
        REFERENCES PRODUCT ( P_ID )
            ON DELETE CASCADE
);
```

In [65]:
df_metadata = pd.DataFrame(metadata)
df_metadata["store"].fillna("Amazon", inplace=True)
df_metadata.head()

/var/folders/d4/h2gdkyfd23g119wvt_vqqv3h0000gn/T/ipykernel_2134/3561490255.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_metadata["store"].fillna("Amazon", inplace=True)


,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],None,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Howard Products,[],"{""Package Dimensions"": ""7.1 x 5.5 x 3 inches; ...",B01CUPMQZE,None,None,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Yes To,[],"{""Item Form"": ""Powder"", ""Skin Type"": ""Acne Pro...",B076WQZGPM,None,None,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],None,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Levine Health Products,[],"{""Manufacturer"": ""Levine Health Products""}",B000B658RI,None,None,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Cherioll,[],"{""Brand"": ""Cherioll"", ""Item Form"": ""Powder"", ""...",B088FKY3VD,None,None,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,None,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Precision,[],"{""UPC"": ""644287689178""}",B07NGFDN6G,None,None,None


In [85]:
df_review = pd.DataFrame(review)
df_review.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
1,5.0,Really nice quality,These are not what I was expecting. They are s...,[],B081GCFHPG,B081GCFHPG,AE5DIA2HDWBPNGBO2FXN2PF4NQJA,1623459734108,0,True
2,5.0,Great product,Great product,[],B07T2L6JQR,B07T2L6JQR,AEEM2QIBMUUXEAJIEQH2WT2KNZZA,1656871648888,0,True
3,5.0,great clipper cleaner,only 2 problems with this product--expensive a...,[],B0008F6QGO,B0008F6QGO,AEITH2MBDGMCSAZSBNAYOEBOJXFA,1402260693000,1,True
4,4.0,Four Stars,I'm very happy with my order. Will order item ...,[],B00JM1BZUW,B00JM1BZUW,AEHVSZ72XXI4SU6XUNXMJAXUYN6Q,1443695835000,0,True


In [97]:
import re

df_product_images = df_metadata
df_product_images = pd.merge(df_product_images,df_review, on="parent_asin", how="right")
df_product_images["p_image"] = df_product_images["images_x"].apply(lambda x: x["hi_res"])
df_product_images = df_product_images.explode("p_image").explode("p_image")
mask = df_product_images["p_image"].apply(lambda x: "http" in str(x))
df_product_images = df_product_images.loc[mask,["parent_asin","p_image"]].drop_duplicates()
df_product_images.rename({"parent_asin":"p_id"},axis=1,inplace=True)

df_product_images

,p_id,p_image
0,B097R46CSY,https://m.media-amazon.com/images/I/810n0gzbOV...
0,B097R46CSY,https://m.media-amazon.com/images/I/71b-eH01ZP...
0,B097R46CSY,https://m.media-amazon.com/images/I/61zB9icdj1...
0,B097R46CSY,https://m.media-amazon.com/images/I/71Ms6rEcHa...
0,B097R46CSY,https://m.media-amazon.com/images/I/61VJQVDrQD...
...,...,...
111293,B09Z29PHVS,https://m.media-amazon.com/images/I/61xxDZqXhl...
111293,B09Z29PHVS,https://m.media-amazon.com/images/I/71-ih42vkd...
111293,B09Z29PHVS,https://m.media-amazon.com/images/I/71p6rVyd-w...
111293,B09Z29PHVS,https://m.media-amazon.com/images/I/71JdsKRtpJ...


In [99]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_product_images.to_sql("product_images", con=conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_product_images)} rows into 'product images'.")

✅ Inserted 186837 rows into 'product images'.


### Daily deals

```sql
-- DAILY_DEALS TABLE
CREATE TABLE DAILY_DEALS (
    P_ID       VARCHAR(10),
    DEAL_DATE  DATE NOT NULL,
    DISCOUNT   NUMERIC NOT NULL,
    PRIMARY KEY ( P_ID,
                  DEAL_DATE ),
    CONSTRAINT DDPRODUCTFK FOREIGN KEY ( P_ID )
        REFERENCES PRODUCT ( P_ID )
            ON DELETE CASCADE
);
```

### carrier
```sql
-- CARRIER TABLE
CREATE TABLE CARRIER (
    CARRIER_ID    INTEGER
        GENERATED BY DEFAULT AS IDENTITY,
    CARRIER_NAME  VARCHAR(30) NOT NULL,
    PRIMARY KEY ( CARRIER_ID )
);
```

### Cart

```sql
-- CART TABLE
CREATE TABLE CART (
    CART_ID      INTEGER
        GENERATED BY DEFAULT AS IDENTITY,
    BUYER_ID     VARCHAR(40),
    TOTAL_QTY    INT DEFAULT 0 NOT NULL CHECK ( TOTAL_QTY >= 0 ),
    TOTAL_PRICE  FLOAT DEFAULT 0 NOT NULL,
    PRIMARY KEY ( CART_ID ),
    CONSTRAINT CBUYERFK FOREIGN KEY ( BUYER_ID )
        REFERENCES BUYER ( BUYER_ID )
            ON DELETE CASCADE
);
```

### Cart items

```sql
-- CART_ITEMS TABLE
CREATE TABLE CART_ITEMS (
    CART_ID  INTEGER,
    P_ID     VARCHAR(10),
    QTY      INTEGER DEFAULT 1 NOT NULL CHECK ( QTY >= 0 ),
    PRIMARY KEY ( CART_ID,
                  P_ID ),
    CONSTRAINT CICARTFK FOREIGN KEY ( CART_ID )
        REFERENCES CART ( CART_ID )
            ON DELETE CASCADE,
    CONSTRAINT CIPRODUCTFK FOREIGN KEY ( P_ID )
        REFERENCES PRODUCT ( P_ID )
            ON DELETE CASCADE
);
```

### Discount

```sql
-- DISCOUNT TABLE
CREATE TABLE DISCOUNT (
    DISCOUNT_ID    INTEGER
        GENERATED BY DEFAULT AS IDENTITY,
    DISCOUNT_NAME  VARCHAR(20) NOT NULL,
    D_DESC         VARCHAR(100) NOT NULL,
    DISCOUNT_AMT   NUMERIC NOT NULL CHECK ( DISCOUNT_AMT >= 0 ),
    PRIMARY KEY ( DISCOUNT_ID )
);
```

### Orders

```sql
-- ORDERS TABLE
CREATE TABLE ORDERS (
    ORDER_ID     INTEGER
        GENERATED BY DEFAULT AS IDENTITY,
    BUYER_ID     VARCHAR(40),
    DISCOUNT_ID  INTEGER,
    PAYMENT_ID   INTEGER,
    ORDER_DATE   DATE NOT NULL,
    PRIMARY KEY ( ORDER_ID ),
    CONSTRAINT OBUYERFK FOREIGN KEY ( BUYER_ID )
        REFERENCES BUYER ( BUYER_ID )
            ON DELETE SET NULL,
    CONSTRAINT OPAYMENTFK FOREIGN KEY ( PAYMENT_ID )
        REFERENCES PAYMENT_DETAILS ( PAYMENT_ID )
            ON DELETE SET NULL,
    CONSTRAINT ODISCOUNTFK FOREIGN KEY ( DISCOUNT_ID )
        REFERENCES DISCOUNT ( DISCOUNT_ID )
            ON DELETE SET NULL
);
```

In [100]:
# --- get from db PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    db_payment_details = pd.read_sql_table("payment_details", con=conn)
    db_customer_payment = pd.read_sql_table("customer_payment", con=conn)
db_payment_details.head()

,payment_id,card_no,cvv,expiry_date,billing_address
0,1,4419335152341009788,763,2027-04-18,3056 Sara Parks Suite 511Port RobinshireNew Yo...
1,2,5149307510258940,311,2029-09-06,548 Mendoza ForgeWest CynthiaArizona62999
2,3,6522716193634794,378,2027-01-26,218 Janet Mission Apt. 082GonzalezviewNew York...
3,4,4874202337238,383,2028-07-27,31935 Chavez Mountains Apt. 665Port PaulmouthW...
4,5,5493877233592579,1126,2029-06-18,97336 Danielle Pass Suite 501North MarkfurtUta...


In [101]:
db_customer_payment.head()

,payment_id,c_id,is_default
0,1,AE222BBOVZIF42YOOPNBXL4UUMYA,1
1,2,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,1
2,3,AE222X475JC6ONXMIKZDFGQ7IAUA,1
3,4,AE222Y4WTST6BUZ4J5Y2H6QMBITQ,1
4,5,AE2232TEZOEWQLAFEX2NA6VBGMYQ,1


In [102]:
customer_payment_merge = pd.merge(db_customer_payment,db_payment_details,on="payment_id")
customer_payment_merge = customer_payment_merge.loc[customer_payment_merge["is_default"]=="1",:]
customer_payment_merge.head()

,payment_id,c_id,is_default,card_no,cvv,expiry_date,billing_address
0,1,AE222BBOVZIF42YOOPNBXL4UUMYA,1,4419335152341009788,763,2027-04-18,3056 Sara Parks Suite 511Port RobinshireNew Yo...
1,2,AE222FP7YRNFCEQ2W3ZDIGMSYTLQ,1,5149307510258940,311,2029-09-06,548 Mendoza ForgeWest CynthiaArizona62999
2,3,AE222X475JC6ONXMIKZDFGQ7IAUA,1,6522716193634794,378,2027-01-26,218 Janet Mission Apt. 082GonzalezviewNew York...
3,4,AE222Y4WTST6BUZ4J5Y2H6QMBITQ,1,4874202337238,383,2028-07-27,31935 Chavez Mountains Apt. 665Port PaulmouthW...
4,5,AE2232TEZOEWQLAFEX2NA6VBGMYQ,1,5493877233592579,1126,2029-06-18,97336 Danielle Pass Suite 501North MarkfurtUta...


In [103]:
df_review.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
1,5.0,Really nice quality,These are not what I was expecting. They are s...,[],B081GCFHPG,B081GCFHPG,AE5DIA2HDWBPNGBO2FXN2PF4NQJA,1623459734108,0,True
2,5.0,Great product,Great product,[],B07T2L6JQR,B07T2L6JQR,AEEM2QIBMUUXEAJIEQH2WT2KNZZA,1656871648888,0,True
3,5.0,great clipper cleaner,only 2 problems with this product--expensive a...,[],B0008F6QGO,B0008F6QGO,AEITH2MBDGMCSAZSBNAYOEBOJXFA,1402260693000,1,True
4,4.0,Four Stars,I'm very happy with my order. Will order item ...,[],B00JM1BZUW,B00JM1BZUW,AEHVSZ72XXI4SU6XUNXMJAXUYN6Q,1443695835000,0,True


In [104]:
df_orders = df_review[["user_id","timestamp"]].rename({"user_id":"buyer_id",
                                                    "timestamp":"order_date"}, axis=1)
df_orders["order_date"] = df_orders["order_date"].apply(lambda x: (pd.to_datetime(x,unit='ms') - 
                                                           pd.Timedelta(days=np.random.randint(3, 8))).date().isoformat())
df_orders

,buyer_id,order_date
0,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,2020-05-13
1,AE5DIA2HDWBPNGBO2FXN2PF4NQJA,2021-06-07
2,AEEM2QIBMUUXEAJIEQH2WT2KNZZA,2022-06-26
3,AEITH2MBDGMCSAZSBNAYOEBOJXFA,2014-06-04
4,AEHVSZ72XXI4SU6XUNXMJAXUYN6Q,2015-09-28
...,...,...
111317,AEHKDBA77OMG72SHVWOXXYRFPJZA,2019-06-24
111318,AEMNUF5IYO7XQKPYXBHGGS642PRA,2020-10-21
111319,AE6IWHJZ5CMNF63W2ZXCDORZT6JQ,2019-02-16
111320,AEAPDTDZCYVGL6FNVOZNMZRSWNTA,2021-04-04


In [105]:
df_orders = pd.merge(df_orders, customer_payment_merge, left_on="buyer_id", right_on="c_id")
df_orders = df_orders[["buyer_id",	"order_date",	"payment_id"]]

In [106]:
# --- Insert into PostgreSQL ---
engine = create_engine(POSTGRES_URI)
with engine.connect() as conn:
    df_orders.to_sql("orders", con=conn, if_exists="append", index=False)
    print(f"✅ Inserted {len(df_orders)} rows into 'orders'.")

✅ Inserted 111322 rows into 'orders'.


### Shipment

```sql
-- SHIPMENT TABLE
CREATE TABLE SHIPMENT (
    SHIPPING_ID           INTEGER
        GENERATED BY DEFAULT AS IDENTITY,
    ORDER_ID              INTEGER,
    P_ID                  VARCHAR(10),
    CARRIER_ID            INTEGER,
    SHIPMENT_TYPE         CHAR(2) DEFAULT 'NP' CHECK ( SHIPMENT_TYPE IN ( 'PP', 'NP' ) ),
    STATUS                VARCHAR(10) NOT NULL,
    EST_DELIVERY_DATE     DATE,
    ACTUAL_DELIVERY_DATE  DATE,
    PRIMARY KEY ( SHIPPING_ID ),
    CONSTRAINT SORDERFK FOREIGN KEY ( ORDER_ID )
        REFERENCES ORDERS ( ORDER_ID )
            ON DELETE SET NULL,
    CONSTRAINT SPRODUCTFK FOREIGN KEY ( P_ID )
        REFERENCES PRODUCT ( P_ID )
            ON DELETE SET NULL,
    CONSTRAINT SCARRIERFK FOREIGN KEY ( CARRIER_ID )
        REFERENCES CARRIER ( CARRIER_ID )
            ON DELETE SET NULL
);
```

### Returns

```sql
-- RETURNS TABLE
CREATE TABLE RETURNS (
    RETURN_ID  INTEGER
        GENERATED BY DEFAULT AS IDENTITY,
    BUYER_ID   VARCHAR(40),
    P_ID       VARCHAR(10),
    ORDER_ID   INTEGER,
    PRIMARY KEY ( RETURN_ID ),
    CONSTRAINT RETBUYERFK FOREIGN KEY ( BUYER_ID )
        REFERENCES BUYER ( BUYER_ID )
            ON DELETE SET NULL,
    CONSTRAINT RETPRODUCTFK FOREIGN KEY ( P_ID )
        REFERENCES PRODUCT ( P_ID )
            ON DELETE SET NULL,
    CONSTRAINT RETORDERFK FOREIGN KEY ( ORDER_ID )
        REFERENCES ORDERS ( ORDER_ID )
            ON DELETE SET NULL
);
```